In [1]:
import itertools
import pandas as pd
import numpy as np
import scipy.stats
from model import BCaus
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge

### Introduction
This notebook reproduces the In-Sample and Out-of-Sample Epsilon-ATE values for the BCAUS DR model reported in Table 1 of our manuscript "Minimizing Bias in Massive Multi-Arm Observational Studies with BCAUS: Balancing Covariates Automatically Using Supervision".
The IHDP benchmark datasets are available online at:
1. Train 672 samples: http://www.fredjo.com/files/ihdp_npci_1-1000.train.npz.zip 
2. Hold-out 75 samples: http://www.fredjo.com/files/ihdp_npci_1-1000.test.npz.zip 

In [2]:
# need to obtain data locally
train_cv = np.load('ihdp_npci_1-1000.train.npz')
test = np.load('ihdp_npci_1-1000.test.npz')

In [3]:
def split_data(data, idx=None):
    """
    This function cleans up the data and optionally splits it into train and cv datasets
    """
    X = data.f.x.copy()
    T = data.f.t.copy()
    YF = data.f.yf.copy()
    YCF = data.f.ycf.copy()
    X[:,13,:]= X[:,13,:]-1  # Because in the raw data this categorical takes values 1,2 instead of 0,1
    if idx is None:
        return X, T, YF, YCF
    else:
        Xtrain = X[:idx,:,:]
        Xcv = X[idx:,:,:]

        Ttrain = T[:idx,:]
        Tcv = T[idx:,:]

        YFtrain = YF[:idx,:]
        YFcv = YF[idx:,:]

        YCFtrain = YCF[:idx,:]
        YCFcv = YCF[idx:,:]
        
        return Xtrain, Ttrain, YFtrain, YCFtrain, Xcv, Tcv, YFcv, YCFcv        

def train_instance(X, T, YF, YCF, bcaus, idx=0):
    """
    This function trains BCAUS and DR Estimators on a single instance of IHDP
    """
    x, t, yf, ycf = X[:,:,idx], T[:,idx], YF[:,idx], YCF[:, idx]
    
    # BCAUS is implemented in the style of a scikit-learn classifier.
    bcaus.fit(x,t)
    
    treated_idx=np.where(t==1)[0]
    control_idx=np.where(t==0)[0]
    
    # Fit estimators for DR
    params={"alpha":[0.001,0.01,0.1]}
    estimator_t = GridSearchCV(Ridge(), param_grid=params, cv=3, n_jobs=3)
    estimator_c = GridSearchCV(Ridge(), param_grid=params, cv=3, n_jobs=3)
    estimator_t.fit(x[treated_idx,:], yf[treated_idx])
    estimator_c.fit(x[control_idx,:], yf[control_idx])
    
    return bcaus, estimator_t, estimator_c

def infer_instance(X, T, YF, YCF, bcaus, estimator_t, estimator_c, idx=0):
    """
    This function infers previously trained BCAUS and DR Estimators on a single instance of IHDP
    """
    x, t, yf, ycf = X[:,:,idx], T[:,idx], YF[:,idx], YCF[:, idx]
    
    scores = bcaus.predict_proba(x)[:,1]
    weights = (t / scores + (1 - t) / (1 - scores))
    
    treated_idx=np.where(t==1)[0]
    control_idx=np.where(t==0)[0]
    
    treatment_yf_pred= estimator_t.predict(x[treated_idx,:])
    treatment_ycf_pred = estimator_c.predict(x[treated_idx,:])
    control_yf_pred = estimator_c.predict(x[control_idx,:])
    control_ycf_pred = estimator_t.predict(x[control_idx,:])

    treatment_ite = (yf[treated_idx]/scores[treated_idx]
                        -treatment_yf_pred*(1-scores[treated_idx])/scores[treated_idx]
                        -treatment_ycf_pred)
    control_ite = control_ycf_pred-(yf[control_idx]/(1-scores[control_idx])
                                    -control_yf_pred*scores[control_idx]/(1-scores[control_idx]))
    
    est_ate = np.mean(np.array(list(treatment_ite)+list(control_ite)))
    
    Y1 = np.array(list(yf[treated_idx]) + list(ycf[control_idx]))
    Y0 = np.array(list(ycf[treated_idx]) + list(yf[control_idx]))
    
    gt_ate = np.mean(Y1)-np.mean(Y0)
    
    return est_ate, gt_ate  

def computeEpsilons(Xtrain, Ttrain, YFtrain, YCFtrain, Xcv, Tcv, YFcv, YCFcv, bcaus):
    """
    This function computes Epsilon-ATE and standard error on train and cv/test sets on all 1000 
    instances of IHDP
    """
    niters = Xtrain.shape[-1]
    
    gt_train=[]  # Ground Truth
    est_train=[] # Estimated
    gt_cv=[]     # Ground Truth
    est_cv=[]    # Estimated

    for fnum in range(niters): 
        
        bcaus, estimator_t, estimator_c = train_instance(Xtrain, Ttrain, YFtrain, YCFtrain, bcaus, idx = fnum)
        
        ate1, gt1 =  infer_instance(Xtrain, Ttrain, YFtrain, YCFtrain, bcaus, estimator_t, estimator_c, idx=fnum)
        est_train.append(ate1)
        gt_train.append(gt1)
        
        ate1, gt1 =  infer_instance(Xcv, Tcv, YFcv, YCFcv, bcaus, estimator_t, estimator_c, idx=fnum)
        est_cv.append(ate1)
        gt_cv.append(gt1)

        if (fnum+1)%1000==0:
            print('Instance %d finished!'%(fnum))
        
    # Train error
    abs_error = np.abs(np.array(gt_train)-np.array(est_train))
    in_eps_ate= np.mean(abs_error)
    in_err = scipy.stats.sem(abs_error)

    # CV error
    abs_error = np.abs(np.array(gt_cv)-np.array(est_cv))
    cv_eps_ate= np.mean(abs_error)
    cv_err = scipy.stats.sem(abs_error)
    
    return in_eps_ate, in_err, cv_eps_ate, cv_err

### Split training data into train and cross validation sets 

In [4]:
Xtrain, Ttrain, YFtrain, YCFtrain, Xcv, Tcv, YFcv, YCFcv = split_data(train_cv, 500)
print(YCFtrain.shape, YCFcv.shape)

(500, 1000) (172, 1000)


### Hyperparameter search

In [5]:
alpha=[0.001, 0.01]
dropout=[0.1, 0.3 ,0.5]
learning_rate_init = [0.001]
nu=[0.0, 0.5, 1, 1.5]

grid_df=pd.DataFrame({"alpha":[],
                     "dropout":[],
                     "learning_rate_init":[],
                     "nu":[],
                     "train_sample_dr":[], 
                     "cv_sample_dr":[]})

for a, d, lr, n in itertools.product(alpha,dropout, learning_rate_init, nu):
    grid_params={"alpha":a, "dropout":d, "learning_rate_init": lr, "nu":n}

    bcaus=BCaus(device='cuda:7', early_stopping=True, eps=1e-07, max_iter=1000, n_iter_no_change=30, **grid_params)
    train_eps_ate, _, cv_eps_ate, _ = computeEpsilons(Xtrain, Ttrain, YFtrain, YCFtrain, Xcv, Tcv, YFcv, YCFcv, bcaus)

    grid_params.update({"train_sample_dr":train_eps_ate, "cv_sample_dr":cv_eps_ate})
    grid_df = grid_df.append(grid_params, ignore_index=True)

Instance 999 finished!
Instance 999 finished!
Instance 999 finished!
Instance 999 finished!
Instance 999 finished!
Instance 999 finished!
Instance 999 finished!
Instance 999 finished!
Instance 999 finished!
Instance 999 finished!
Instance 999 finished!
Instance 999 finished!
Instance 999 finished!
Instance 999 finished!
Instance 999 finished!
Instance 999 finished!
Instance 999 finished!
Instance 999 finished!
Instance 999 finished!
Instance 999 finished!
Instance 999 finished!
Instance 999 finished!
Instance 999 finished!
Instance 999 finished!


In [6]:
grid_df = grid_df.sort_values('cv_sample_dr').reset_index(drop = True)
best_params = grid_df.iloc[0,:4].to_dict()

### Evaluate on hold-out set

In [7]:
X_in, T_in, YF_in, YCF_in = split_data(train_cv)
X_oo, T_oo, YF_oo, YCF_oo = split_data(test)

bcaus=BCaus(device='cuda:7', early_stopping=True, eps=1e-07, max_iter=1000, n_iter_no_change=30, **best_params)
in_eps_ate, in_err, oo_eps_ate, oo_err = computeEpsilons(X_in, T_in, YF_in, YCF_in, X_oo, T_oo, YF_oo, YCF_oo, bcaus)

Instance 999 finished!


In [8]:
print('In-Sample Epsilon ATE = {} +/- {}'.format(in_eps_ate, in_err))
print('Out-of-Sample Epsilon ATE = {} +/- {}'.format(oo_eps_ate, oo_err))

In-Sample Epsilon ATE = 0.13212367321365898 +/- 0.004395004517373404
Out-of-Sample Epsilon ATE = 0.29538274983277246 +/- 0.011989530664810202
